First we need to search all related literature that might include data or information of thalamocortical connections.

In [460]:
# import required packages
import pandas as pd
from urllib.request import Request, urlopen
import urllib.request
import re
from bs4 import BeautifulSoup 
import requests
import time
import os.path
import urllib.request
import csv
# from scholarly import scholarly

In the next cell, we present all parameters that might have an effect on the search results, including:<br>
1. searching keyword lexicon
2. on-topic keyword lexicon
3. academic databases
4. seed papers
5. connectome database and 
6. connectome database queries
7. ChatPDF queries for relatedness of topic

In [461]:
# parameters

# searching keyword lexicon
# (macaque OR Macaque) AND (thalamocortical OR thalamocortical OR corticothalamic OR 'cortico-thalamic' OR thalamus OR cortex)
search_kws_lexicon = 'macaque AND (thalamus OR cortex OR thalamocortical OR thalamo-cortical OR corticothalamic OR cortico-thalamic)'

# on-topic keyword lexicon
on_topic_kws = ['thalamocortical', 'thalamo-cortical', 'corticothalamic', 'cortico-thalamic',
                'tracing', 'tracer', 'tract tracing', 'tract-tracing', 'axonal tracing', 'neural anatomical tracing', 
                'connection', 'projection', 'connectivity', 'connectome', 
                'thalamus', 'cortex']

# academic databases
Semantic_Scholar = 'https://www.semanticscholar.org/'
Google_Scholar = 'https://scholar.google.com/'
# arXiv = 'https://arxiv.org/'
Web_of_Science = 'https://www.webofscience.com/wos/woscc/basic-search' # can be expoted to excel file
PubMed_Central_PMC = 'https://www.ncbi.nlm.nih.gov/pmc/' # can be exported to .csv file and abstract.txt file
Europe_PMC = 'https://europepmc.org/' # can be exported to .csv file or abstract and full open access file .xml
acad_dbs = [Semantic_Scholar, Google_Scholar, Web_of_Science, PubMed_Central_PMC, Europe_PMC]

# seed papers specification
seed_papers = []

# connectome database and queries specification
# we search the CoCoMac
connec_db = 'http://cocomac.g-node.org/sitemap/index.php?path=CoCoMac|Search'
connec_db_quries = []

# ChatPDF, queries for relatedness of topic
ChatPDF_related_queries = []

In [ ]:
# all the file paths and request headers
path_urls = '/Users/didihou/myProjects/didihou_master_project/list_of_literature_urls.txt'
path_potential = '/Users/didihou/myProjects/didihou_master_project/list_of_potential_related_literature.csv'
path_related_urls = '/Users/didihou/myProjects/didihou_master_project/list_of_related_literature.txt'
pdf_folder_path = '/Users/didihou/myProjects/liter_pdfs'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

In [462]:
from serpapi import GoogleSearch
params = {
  "engine": "google_scholar",
  "q": "macaque thalamus OR thalamocortical OR thalamo-cortical",
  "api_key": "c397fc278e5a3483e64735af275a21bf6f78b17a8084ea7f507d933bb4d4b90a"
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]

In [ ]:
organic_results

In [ ]:
# method 1: searching academic databases using searching keyword lexicon
def search_acad_db(keywords, acad_db):
    return

In [ ]:
# method 2: spanning citations of seed papers
def expand_citation(seed_papers):
    return

In [ ]:
# method 3: querying the existing connectome database 
def search_conne_db(connec_db, connec_db_quries):
    return 

In [ ]:
# searching for related literature and write them into file 'list_of_potential_related_literature.txt'
def record_liter(doi_or_url, path):
    with open(path, 'a+') as url_file:
        url_file.seek(0)
        # If file is not empty then append '\n'
        data = url_file.read(100)
        if len(data) > 0 :
            url_file.write('\n')
        # Append text at the end of file
        url_file.write(doi_or_url)
        
'''
# code for testing function: record_liter(doi_or_url, path)
doi_or_url = 'http://cocomac.g-node.org/main/index.php'
for i in range(10):
    record_liter(doi_or_url)
'''

In [ ]:
# read line by line the urls of related literature in the tex file and access the content
# seach for on-topic keywords and record the number of times they appear and record to a .csv file
def scan_poten_urls(path_urls, on_topic_kws):
    with open(path_urls, 'r') as url_file:
        for line in url_file:
            # Process each line here
            for i in range(10):
                response = requests.get(line)
                if response.status_code == 200:
                    content = response.text
                    # process the content of the webpage here
                    print(content)
                    # code
                    content = response.text
            if(i == 9):
                print("Error accessing the webpage. Status code:", response.status_code)
                 

# code for testing function: scan_poten_urls(path_urls, on_topic_kws)
line = 'https://onlinelibrary.wiley.com/doi/epdf/10.1002/cne.902570211'
for i in range(10):
    req = Request(
    url=line, 
    headers={'User-Agent':'Mozilla/5.0'})
    webpage = urlopen(req).read()
    response = requests.get(line)
    if response.status_code == 200:
        content = response.text
        # process the content of the webpage here
        break
if(i == 9):
    print("Error accessing the webpage. Status code:", response.status_code)
else:
    print("Success")

In [ ]:
# web scraping
# URL from which pdfs to be downloaded
url = "https://medium.com/analytics-vidhya/webscraping-a-site-with-pagination-using-beautifulsoup-fa0a09804445"

# Requests URL and get response object
response = requests.get(url)

# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')

# Find all hyperlinks present on webpage
links = soup.find_all('a')

i = 0

# From all links check for pdf link and
# if present download file
for link in links:
    print(link)
    if ('.pdf' in link.get('href', []) or 'pdf.' in link.get('href', [])):
        i += 1
        print("Downloading file: ", i)

        # Get response object for link
        response = requests.get(link.get('href'))

        # Write content in pdf file
        pdf = open("pdf"+str(i)+".pdf", 'wb')
        pdf.write(response.content)
        pdf.close()
        print("File ", i, " downloaded")

print("All PDF files downloaded")

In [510]:
def search_acad_dbs(acad_db_name, path_urls):
    if acad_db_name == 'Google Scholar':
        url = 'https://scholar.google.com/scholar?start=0&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5'
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content,'lxml')
        num_results_str = soup.find_all('div', {'class': 'gs_ab_mdw'})[1].get_text().split()[1]
        # print(int(num_results_str))
        num_results = int(re.sub(r'[^\w\s]', '', num_results_str))
        pages = int(num_results/10)
        pages = 10
        # print(pages)
        # search all pages
        for page in range(pages):
            time.sleep(2)
            start = page * 10
            # google scholar
            page_url = 'https://scholar.google.com/scholar?start=' + str(start) + '&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5'
            # search a page
            response = requests.get(page_url, headers = headers)
            # print(url)
            soup = BeautifulSoup(response.content,'lxml') 
            # print(soup.select('[data-lid]')) 
            for item in soup.select('[data-lid]'): 
                try: 
                    with open(path_urls, 'a+') as url_file:
                        # append text at the end of file
                        url_file.write(item.select('h3')[0].find_all('a', href=True)[0]['href'])
                        url_file.write('\n')
                except Exception as e: 
                    #raise e
                    print("error")
    elif acad_db_name == 'PubMed':
        url = 'https://pubmed.ncbi.nlm.nih.gov/?term=macaque%20AND%20(thalamus%20OR%20cortex%20OR%20thalamocortical%20OR%20thalamo-cortical%20or%20corticothalamic%20OR%20cortico-thalamic)&page=1'
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.content,'lxml')
        # print(soup)
        num_results_str = soup.find_all('span', {'class': 'value'})[0].get_text()
        print(num_results_str)
        num_results = int(re.sub(r'[^\w\s]', '', num_results_str))
        pages = int(num_results/10)
        print(pages)
    else:
        None
    
    '''
    for page in range(pages):
        time.sleep(2)
        start = page * 10
        # google scholar
        url = 'https://scholar.google.com/scholar?start=' + str(start) + '&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5'
        # pubmed
        url = 'https://pubmed.ncbi.nlm.nih.gov/?term=macaque%20AND%20(thalamus%20OR%20cortex%20OR%20thalamocortical%20OR%20thalamo-cortical%20or%20corticothalamic%20OR%20cortico-thalamic)&page=1
        response = requests.get(url,headers = headers)
        # print(url)
        soup = BeautifulSoup(response.content,'lxml') 
        #print(soup.select('[data-lid]')) 
        for item in soup.select('[data-lid]'): 
            try: 
                # print('----------------------------------------') 
                # print(item)  
                # print(item.select('h3')[0])
                with open(path_urls, 'a+') as url_file:
                    url_file.seek(0)
                    # If file is not empty then append '\n'
                    data = url_file.read(100)
                    if len(data) > 0 :
                        url_file.write('\n')
                        # Append text at the end of file
                    url_file.write('----------------------------------------\n')
                    url_file.write(item.select('h3')[0].get_text())
                    url_file.write('\n')
                    # print(item.select('h3')[0].get_text())
                    for a in item.select('h3')[0].find_all('a', href=True):
                        # print(a['href'])
                        url_file.write(a['href'])
                        url_file.write('\n')
                        # print(item.select('a'))
                        # print("PDF link:")
                    url_file.write(item.select('a')[0]['href'])
                    url_file.write('\n')
                    # print(item.select('a')[0]['href'])
                    # print(item.select('.gs_rs')[0].get_text()) 
                    # print('----------------------------------------') 
            except Exception as e: 
                #raise e 
                print('')
    '''
    
# test code
acad_db_name = 'Google Scholar'
search_acad_dbs(acad_db_name, path_urls)

In [463]:
# count the number of times that certain on-topic keyword appear in a given text
def count_keyword(text: str, keyword: str) -> int:
    # print(text)
    word_count = 0
    for word in text.strip().split(" "):
        # print(word)
        if word == keyword:
            word_count += 1
    # print(f"I found {word_count} words")
    return word_count

'''
# test code
text = 'This apple 6i7s very tasty？、  2but th&e banana is not delicious at all.6'
keyword = 'is'
count = count_keyword(text, keyword)
print(count)
'''

"\n# test code\ntext = 'This apple 6i7s very tasty？、  2but th&e banana is not delicious at all.6'\nkeyword = 'is'\ncount = count_keyword(text, keyword)\nprint(count)\n"

In [464]:
# download pdf given pdf_url
def download_pdf_file(pdf_url: str, pdf_folder_path: str, file_name: str) -> bool:
    # param url: The url of the PDF file to be downloaded
    # return: True if PDF file was successfully downloaded, otherwise False.
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'} 
    response = requests.get(pdf_url, stream=True, headers = headers)
    
    # download the .pdf file to the pdf_file_path folder
    # write content in pdf file
    pdf_path = os.path.join(pdf_folder_path, file_name)
    if response.status_code == 200:
        # save in current working directory
        with open(pdf_path, 'wb') as pdf_object:
            pdf_object.write(response.content)
            print(f'{file_name} was successfully saved!')
            return True
    else:
        print(f'Uh oh! Could not download {file_name},')
        print(f'HTTP response status code: {response.status_code}')
        return False

In [465]:
# read line by line the urls of related literature in the tex file and access the content
# seach for on-topic keywords and record the number of times they appear and record to a .csv file
def ChatPDF_relatedness(path_urls, ChatPDF_related_queries):
    with open(path_urls, 'r') as url_file:
        for line in url_file:
            a = 1
            # code
            
'''
# code for testing function: ChatPDF_relatedness(path_urls, ChatPDF_related_queries)

'''

'\n# code for testing function: ChatPDF_relatedness(path_urls, ChatPDF_related_queries)\n\n'

In [466]:
# extract text of title, abstract, keywords, introduction from given url and download PDF of the paper
# return a json object consisting: DOI, url, keyword:frequency pair, relatedness of answers from ChatGPT

def count_freq_from_liter(url, on_topic_kws):
    print(url)
    # access the url by web scraping
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'} 
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.content,'lxml')
    print(len(soup))
    
    # extract DOI
    # print(soup.find_all("a", {'class': 'id-link'}, href = True)[1]['href'])
    doi = soup.find_all("a", {'class': 'id-link'}, href = True)[1]['href']
    
    # extract title
    title = soup.select('h1')[0].get_text().strip()
    title = re.sub(' +', ' ', title).capitalize()
    
    '''
    # extract PDF link if exists
    print(doi)
    response_pdf = requests.get(doi, headers = headers)
    print(response_pdf.url)
    pdf_page_link = response_pdf.url
        
    # pdf_page = soup.find_all("a", {'class':'link-item dialog-focus'}, href = True)[0]['href']
    
    # print(pdf_page_link)
    pdf_page = requests.get(pdf_page_link, headers = headers)
    soup_pdf = BeautifulSoup(pdf_page.content,'lxml')
    print(len(soup_pdf.find_all("a", href = True)))
    pdf_link = soup_pdf.find_all("a", href = True)[0]['href']
    
    
    # print(pdf_link)
    pdf_link = 'https://www.ncbi.nlm.nih.gov' + pdf_link
    '''
    # extract title, abstract, keywords, introduction from the returned html file
    # count keywords from abstract + keywords
    abs_kws = soup.find_all("div", {'class': 'abstract'})[0].get_text()
    abs_kws = abs_kws.strip()
    abs_kws = re.sub(' +', ' ', abs_kws)
    text = title + ' ' + abs_kws
    text = re.sub(r"[^a-zA-Z' ']","",text).lower()
    
    # record the information into json
    info_json = {}
    info_json['DOI'] = doi,
    info_json['url'] = url,
    info_json['title'] = title
    # info_json['pdf_link'] = pdf_link
    # count the on-topic keywords or calculate the frequency
    for i in range(len(on_topic_kws)):
        word_count = count_keyword(text, on_topic_kws[i])
        info_json[on_topic_kws[i]] = word_count
    
    return info_json
  
'''
# test
url = 'https://pubmed.ncbi.nlm.nih.gov/32053769/'
pdf_folder_path = '/Users/didihou/myProjects/liter_pdfs'
info_json_ele = count_freq_from_liter(url, on_topic_kws)
file_name = 'test.pdf'
download_pdf_file(info_json_ele['pdf_link'], pdf_folder_path, file_name)
print(info_json_ele)
'''

"\n# test\nurl = 'https://pubmed.ncbi.nlm.nih.gov/32053769/'\npdf_folder_path = '/Users/didihou/myProjects/liter_pdfs'\ninfo_json_ele = count_freq_from_liter(url, on_topic_kws)\nfile_name = 'test.pdf'\ndownload_pdf_file(info_json_ele['pdf_link'], pdf_folder_path, file_name)\nprint(info_json_ele)\n"

In [467]:
# test the redirect of the urls
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'} 
'''
response_pdf = requests.get('https://doi.org/10.1016/j.neuron.2020.01.005', allow_redirects=True, headers = headers)
print(response_pdf.history)
print(response_pdf.url)
response_pdf_1 = requests.get('https://linkinghub.elsevier.com/retrieve/pii/S0896627320300052', allow_redirects=True, headers = headers)
print(response_pdf_1.history)
print(response_pdf_1.url)


response_pdf = requests.get('https://onlinelibrary.wiley.com/doi/10.1111/ejn.13910', headers = headers)
soup_pdf = BeautifulSoup(response_pdf.content,'lxml')
print(soup_pdf)
'''

"\nresponse_pdf = requests.get('https://doi.org/10.1016/j.neuron.2020.01.005', allow_redirects=True, headers = headers)\nprint(response_pdf.history)\nprint(response_pdf.url)\nresponse_pdf_1 = requests.get('https://linkinghub.elsevier.com/retrieve/pii/S0896627320300052', allow_redirects=True, headers = headers)\nprint(response_pdf_1.history)\nprint(response_pdf_1.url)\n\n\nresponse_pdf = requests.get('https://onlinelibrary.wiley.com/doi/10.1111/ejn.13910', headers = headers)\nsoup_pdf = BeautifulSoup(response_pdf.content,'lxml')\nprint(soup_pdf)\n"

In [468]:
# automatic filtering
def auto_filter(df_potencial_liter):
    return 

In [469]:
# manual filtering
# manual filtering is performed by reading the tilte, abstract, keywords, introduction section of each publication 
# in the order of sorted list by the possiblity of relatedness
def manual_filter(path_potential):
    # read and munual confirm the related literature
    return

In [473]:
# add the information to list_of_potential_related_literature.csv
def add_rows_to_csv(path_potential, info_json, columns):
    new_row = info_json
    df_new_row = pd.DataFrame(new_row, columns = columns)
    
    with open(path_potential, 'r') as csvfile:
        csv_dict = [row for row in csv.DictReader(csvfile)]
        if len(csv_dict) == 0:
            df_new_row.to_csv(path_potential, index = False, header = True)
        else:
            df_new_row.to_csv(path_potential, mode = 'a', index = False, header = False)
    # df_new_row.to_csv(path_potential, index = False)
    # df_liter = df_liter.append(new_row, ignore_index=False)
    # print(df_liter)

In [474]:
# scan url in list_of_literature_urls.txt and record information and download pdf
def scan_record_download(path_urls, on_topic_kws, pdf_folder_path, columns):
    file_index = 0
    with open(path_urls, 'r') as url_file:
        for url in url_file:
            # print(url)
            info_json = {}
            info_json = count_freq_from_liter(url.strip(), on_topic_kws)
            add_rows_to_csv(path_potential, info_json, columns)
            # download_pdf_file(info_json['pdf_link'], pdf_folder_path, str(file_index))
            file_index += 1

In [475]:
# main program
'''
# clear all files
with open(path_urls, 'w') as f:
    f.truncate()
    f.close()
    
with open(path_related_urls, 'w') as f:
    f.truncate()
    f.close()
'''
# search for potentially related literature using the listed 3 methods
# method 1
# search_acad_db(keywords, acad_db)
columns = ['DOI', 'url', 'title'] + on_topic_kws
with open(path_potential, 'w') as f:
    f.truncate()
    f.close()

scan_record_download(path_urls, on_topic_kws, pdf_folder_path, columns)

'''
# method 2
liter_list_2 = []
# liter_list_2 = expand_citation(seed_papers)
for doi_or_url in liter_list_2:
    record_liter(doi_or_url, path_urls)

# method 3
liter_list_3 = []
# liter_list_3 = search_conne_db(connec_db, connec_db_quries)
for doi_or_url in liter_list_3:
    record_liter(doi_or_url, path_urls)
'''

# eleminate duplications
# code

# scan all the potential related literature and searching for the on-topic keywords and 
# record the number of times they appear
# scan_poten_urls(path_urls, on_topic_kws)

# send .PDF publication of all potential related literatures to ChatPDF.con and ask for relatedness 
# then record the answer to the list_of_potential_related_literature.csv as well
#ChatPDF_relatedness(path_urls, chatpdf_related_queries)

# now we have a list of potential related literature and the information about relatedness 
# stored in the file "list_of_potential_related_literature.csv"
# now we may perform a automatic filtering and manual filtering of the literature

# automatic filtering
#auto_filter(path_potential)

# manual filtering
# manual_filter(path_potential, path_related_urls)

https://pubmed.ncbi.nlm.nih.gov/32053769/
2
https://pubmed.ncbi.nlm.nih.gov/34524542/
2
https://pubmed.ncbi.nlm.nih.gov/35851953/
2
https://pubmed.ncbi.nlm.nih.gov/29542210/
2


'\n# method 2\nliter_list_2 = []\n# liter_list_2 = expand_citation(seed_papers)\nfor doi_or_url in liter_list_2:\n    record_liter(doi_or_url, path_urls)\n\n# method 3\nliter_list_3 = []\n# liter_list_3 = search_conne_db(connec_db, connec_db_quries)\nfor doi_or_url in liter_list_3:\n    record_liter(doi_or_url, path_urls)\n'

This is the end of semi-automated literature search.

Now we have a list of actually related literature stored in list_of_related_literature.txt

Next step: we perform a information search on the list of related literature, see the notebook named "semi_automated_information_search.ipynb"